In [87]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader


# import for NLP
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from keras.utils import to_categorical



In [88]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [89]:
# mount the drive where your dataset is available
from google.colab import drive
drive.mount('/content/drive')
filepath='/content/drive/MyDrive/datasets/multimodal_product_classification/' # add your own path. Where to save the dataset

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [90]:
# Load data
X_train = pd.read_csv(filepath+'X_train.csv')
y_train = pd.read_csv(filepath+'Y_train.csv')
X_train=X_train.drop(columns="Unnamed: 0")
y_train=y_train.drop(columns="Unnamed: 0")

In [91]:
# Cleaning and Preprocessing Text
def clean_text(text):
    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-ZäöüßÄÖÜ ]', '', text)
    # Convert text to lowercase
    text = text.lower()
    return text

In [92]:
# Apply cleaning function to the 'designation' column
X_train['designation'] = X_train['designation'].fillna('').apply(clean_text)

# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train['designation'])
sequences = tokenizer.texts_to_sequences(X_train['designation'])

# Padding to max length of text
data = pad_sequences(sequences, maxlen=34)

# Assuming the number of unique words in the tokenizer plus 1 is vocab_size
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)# nearly 70 000 of vocab size, it seems too much




69189


In [93]:
# Split data into training and validation set (80% train, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(data, y_train, test_size=0.2,shuffle=False)
y_train=y_train["prdtypecode"].tolist()
y_val=y_val["prdtypecode"].tolist()

In [94]:
class TextDataset(Dataset):
    def __init__(self, X, Y):
        self.inputs = X
        self.labels = Y

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        x = torch.tensor(self.inputs[idx])
        y = torch.tensor(self.labels[idx], dtype=torch.int)
        return x, y

In [95]:
train_loader=TextDataset(X_train,y_train)

In [96]:
val_loader=TextDataset(X_val,y_val)

# Model definition


In [97]:

# Define the model
class CNN_classifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, num_classes):
        super(CNN_classifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.conv_blocks = nn.ModuleList([
            nn.Conv2d(1, 512, (i, embedding_dim), padding=(i-1, 0))
            for i in range(1,7)
        ])
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(512 * 6, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        x = x.unsqueeze(1)  # Add channel dimension for Conv2d
        conv_outputs = [nn.functional.relu(conv_block(x)).max(dim=3)[0] for conv_block in self.conv_blocks]
        x = torch.cat(conv_outputs, dim=1)
        # Flatten Layer
        x = x.view(x.size(0), -1)
        # Dropout Layer
        x = self.dropout(x)
        # Dense Layer
        x = self.fc(x)
        return x


In [98]:

# Initialize the model
embedding_dim = 300
num_classes = 27
max_designation_len=34
model = CNN_classifier(max_designation_len, embedding_dim, num_classes)

# Convert the model to CUDA if available
model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())
from sklearn.metrics import f1_score
f1_metric = lambda preds, labels: f1_score(labels, torch.argmax(preds, dim=1), average='macro')


# Print the model summary
print(model)


CNN_classifier(
  (embedding): Embedding(34, 300)
  (conv_blocks): ModuleList(
    (0): Conv2d(1, 512, kernel_size=(1, 300), stride=(1, 1))
    (1): Conv2d(1, 512, kernel_size=(2, 300), stride=(1, 1), padding=(1, 0))
    (2): Conv2d(1, 512, kernel_size=(3, 300), stride=(1, 1), padding=(2, 0))
    (3): Conv2d(1, 512, kernel_size=(4, 300), stride=(1, 1), padding=(3, 0))
    (4): Conv2d(1, 512, kernel_size=(5, 300), stride=(1, 1), padding=(4, 0))
    (5): Conv2d(1, 512, kernel_size=(6, 300), stride=(1, 1), padding=(5, 0))
  )
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=3072, out_features=27, bias=True)
)


In [99]:
batch_size=32
epochs=10

In [100]:
# Convert labels to categorical
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)

# Convert to one-hot encoding
y_train_categorical = to_categorical(y_train_encoded)
y_val_categorical = to_categorical(y_val_encoded)


In [101]:
# Train the model
num_epochs = 1  # Adjust as needed
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        val_preds = []
        val_labels = []
        for val_inputs, val_labels_batch in val_loader:
            val_inputs, val_labels_batch = val_inputs.to(device), val_labels_batch.to(device)
            val_outputs = model(val_inputs)
            val_preds.append(val_outputs.cpu())
            val_labels.append(val_labels_batch.cpu())

    val_preds = torch.cat(val_preds, dim=0)
    val_labels = torch.cat(val_labels, dim=0)

    val_f1 = f1_metric(val_preds, val_labels)

    print(f"Epoch {epoch + 1}/{num_epochs}, Validation F1 Score: {val_f1:.4f}")


IndexError: ignored